<a href="https://colab.research.google.com/github/krishnasd1007/sea_ice/blob/main/seaiceknn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# **DataEngineering**

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

path = '/gdrive/My Drive/sea_ice/'
data = pd.read_csv(path + 'DRIFT_DATA_TRAIN.csv')

data['Ui'] = np.sqrt((data['u_buoy']*data['u_buoy'])+(data['v_buoy']*data['v_buoy']))
data['Uw'] = np.sqrt((data['x_EASE']*data['x_EASE'])+(data['y_EASE']*data['y_EASE']))
data['Ua'] = np.sqrt((data['u_ERA5']*data['u_ERA5'])+(data['v_ERA5']*data['v_ERA5']))
#data['Uiwx'] = data['u_buoy'] - data['x_EASE']
#data['Uiwy'] = data['v_buoy'] - data['y_EASE']
#data['Uiw'] = np.sqrt((data['Uiwx']*data['Uiwx'])+(data['Uiwy']*data['Uiwy']))
#data['dot'] = (data['u_buoy']*data['Uiwx']) + (data['v_buoy']+data['Uiwy'])
#data['denom'] = data['Ua'] * data['Uiw']
#data['th'] = data['dot'] / data['denom']
#data['th'] = (2*(data['th'] - data['th'].min()) / (data['th'].max() - data['th'].min())) - 1 
#data['theta'] = (np.arccos(data['th'])) * (180/np.pi)
data = data.drop(columns=['year','month','day','doy','h_cs2smos','id_buoy','u_buoy','v_buoy'])
data.head(2)

,x_EASE,y_EASE,u_ERA5,v_ERA5,sic_CDR,h_piomas,d2c,Ui,Uw,Ua
0,147.506958,138.582672,-6.704156,-0.321260,0.990195,3.189743,522.523298,1.370671,202.394317,6.711849
1,146.834778,120.509880,-6.818630,-0.674205,0.966372,2.484009,412.767669,0.741408,189.955477,6.851881


In [4]:
data.isnull().sum(axis = 0)

x_EASE      0
y_EASE      0
u_ERA5      0
v_ERA5      0
sic_CDR     0
h_piomas    0
d2c         0
Ui          0
Uw          0
Ua          0
dtype: int64

# **Model for speed**

In [5]:
x = data.drop(columns = ['Ui'])
y = data['Ui']

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

x = scaler.fit_transform(x)
x = pd.DataFrame(x)

In [7]:
from sklearn.neighbors import KNeighborsRegressor

model_speed = KNeighborsRegressor(n_neighbors=2)

model_speed.fit(x,y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [8]:
from sklearn.metrics import mean_squared_error 

pred_y = model_speed.predict(x)
mse =mean_squared_error(y, pred_y)
me = np.mean(pred_y - y)
print("Mean Error:",me)
print("Mean Squared Error:",mse)

Mean Error: -0.07445523565285318
Mean Squared Error: 7.110207604500074


In [9]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 2.666497253795712


#**Test set prediction**

In [10]:
datatest = pd.read_csv(path + 'DRIFT_DATA_TEST_hackathon.csv')

datatest['Uw'] = np.sqrt((datatest['x_EASE']*datatest['x_EASE'])+(datatest['y_EASE']*datatest['y_EASE']))
datatest['Ua'] = np.sqrt((datatest['u_ERA5']*datatest['u_ERA5'])+(datatest['v_ERA5']*datatest['v_ERA5']))
datatest['Ui'] = np.sqrt((datatest['u_buoy']*datatest['u_buoy'])+(datatest['v_buoy']*datatest['v_buoy']))

datatest = datatest.drop(columns=['year','month','day','doy','h_cs2smos','id_buoy','u_buoy','v_buoy'])

In [11]:
x = datatest.drop(columns=['Ui'])
y = datatest['Ui']

In [12]:
from sklearn.neighbors import KNeighborsRegressor

model_speed = KNeighborsRegressor(n_neighbors=2)

model_speed.fit(x,y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [13]:
from sklearn.metrics import mean_squared_error 

pred_y = model_speed.predict(x)
mse =mean_squared_error(y, pred_y)
me = np.mean(pred_y - y)
print("Mean Error:",me)
print("Mean Squared Error:",mse)

Mean Error: -0.34033835206706053
Mean Squared Error: 10.838290612447924


In [14]:
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 3.292155921648901
